#  Strategy Enhancements & Sensitivity Analysis
This notebook introduces enhancements to the base model strategies (from 06_Full_Strategy_Clean_Baseline) including:
- Holding period constraints
- Volatility-based sizing
- Stop-loss logic
- Transaction cost robustness testing

Each enhancement will be evaluated independently and cumulatively.

## 1. Setup & Imports

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import vectorbt as vbt

In [ ]:
# Define consistent model colors (run this once near the top of your notebook)
model_colors = {
    "SPY (Benchmark)": "#1f77b4",
    "LSTM": "#ff7f0e",
    "GRU": "#2ca02c",
    "CNN-LSTM": "#d62728",
    "ATT-LSTM": "#9467bd",
    "Transformer": "#8c564b"
}

## 2. Load Processed Model Outputs & SPY Prices
- Ensure alignment with the prediction horizon.
- Only include models from `df_lstm.csv`, etc.


In [ ]:
# === Load SPY ===
df_spy = pd.read_csv("../data/GSPC_fixed.csv")
df_spy['Date'] = pd.to_datetime(df_spy['Date'])
df_spy.set_index('Date', inplace=True)
df_spy = df_spy[['Adjusted_close']].rename(columns={'Adjusted_close': 'SPY'})

# === Load Model Predictions ===
df_lstm = pd.read_csv('../data/df_lstm.csv')
df_gru = pd.read_csv('../data/df_gru.csv')
df_cnn = pd.read_csv('../data/df_cnn.csv')
df_att = pd.read_csv('../data/df_att.csv')
df_trans = pd.read_csv('../data/df_trans.csv')

# === Assign Synthetic Business Day Index ===
date_index = pd.date_range(start='2018-12-28', periods=len(df_lstm), freq='B')
for df in [df_lstm, df_gru, df_cnn, df_att, df_trans]:
    df['date'] = date_index
    df.set_index('date', inplace=True)

# === Align SPY ===
end_date = df_lstm.index[-1]
spy_prices = df_spy['SPY'].loc['2018-12-28':end_date]


## 3. Define Core Strategy Functions

In [ ]:
# Generate binary signal
def generate_signals(df, threshold=0.5):
    return (df['predictions'] > threshold).astype(int)

# Basic backtest
def run_vectorbt_strategy(df, signal, price_series, fees=0.001, size=1.0):
    return vbt.Portfolio.from_signals(
        close=price_series,
        entries=signal == 1,
        exits=signal == 0,
        size=size,
        freq='1D',
        init_cash=100,
        fees=fees
    )


## 4. Enhancement: Apply Minimum Holding Period
Define modular strategy logic that supports optional features:
- `min_holding_days`
- `volatility_adjustment`
- `stop_loss_pct`

In [ ]:
# Enforce minimum holding period (e.g., 3 days)
def apply_min_holding(signal_series, min_hold=3):
    signal = signal_series.copy()
    current = signal.iloc[0]
    counter = 0
    for i in range(1, len(signal)):
        if counter > 0:
            signal.iloc[i] = current
            counter -= 1
        elif signal.iloc[i] != current:
            current = signal.iloc[i]
            counter = min_hold - 1
            signal.iloc[i] = current
    return signal


## 5. Execute Strategies with Holding Constraint

In [ ]:
model_dfs = {
    'LSTM': df_lstm.copy(),
    'GRU': df_gru.copy(),
    'CNN-LSTM': df_cnn.copy(),
    'ATT-LSTM': df_att.copy(),
    'Transformer': df_trans.copy()
}

hold_portfolios = {}

for name, df in model_dfs.items():
    raw_signal = generate_signals(df)
    held_signal = apply_min_holding(raw_signal, min_hold=3)

    # Align SPY to model index to avoid shape mismatch
    aligned_spy = spy_prices.reindex(df.index)


    pf = run_vectorbt_strategy(df, held_signal, aligned_spy)
    hold_portfolios[name] = pf



# Add SPY Buy & Hold to hold_portfolios
aligned_spy = spy_prices.reindex(df_lstm.index)  # or any model index
spy_pf = vbt.Portfolio.from_signals(
    close=aligned_spy,
    entries=pd.Series(True, index=aligned_spy.index),
    exits=pd.Series(False, index=aligned_spy.index),
    freq='1D',
    init_cash=100,
    fees=0.0
)
hold_portfolios["SPY (Benchmark)"] = spy_pf
    

## 6. Plot Strategy Comparison

In [ ]:
# --- Plot ---
plt.figure(figsize=(12, 6))

for name, pf in hold_portfolios.items():
    model_key = name.replace(" + MinHold", "")
    color = model_colors.get(model_key, "#333333")  # fallback to gray
    label = name if name == "SPY (Benchmark)" else f"{model_key} + MinHold"
    plt.plot(pf.value(), label=label, linewidth=2.5, alpha=0.9, color=color)

plt.title("Model Strategies with Minimum Holding Period Constraint", fontsize=16, fontweight='bold')
plt.xlabel("Date", fontsize=14, fontweight='bold')
plt.ylabel("Portfolio Value (Base 100)", fontsize=14, fontweight='bold')

# Bold tick labels
plt.xticks(fontsize=11, fontweight='bold')
plt.yticks(fontsize=11, fontweight='bold')

# Legend formatting
legend = plt.legend(fontsize=12, loc='best', frameon=True)
for text in legend.get_texts():
    text.set_fontweight('bold')

plt.grid(True, linestyle='--', alpha=0.7)
plt.tight_layout()
plt.savefig("1_min_holding_comparison.png", dpi=150)
plt.show()


## 7. Summary Table

In [ ]:
summary = pd.DataFrame()

for name, pf in hold_portfolios.items():
    stats = {
        'Final Value': pf.value().iloc[-1],
        'Total Return [%]': pf.total_return() * 100,
        'Sharpe Ratio': pf.sharpe_ratio(),
        'Max Drawdown [%]': pf.max_drawdown() * 100,
        'Trades': pf.stats()['Total Trades']
    }
    summary[name] = pd.Series(stats)

summary = summary.T.round(2)
display(summary.sort_values('Sharpe Ratio', ascending=False))


## 8: Volatility-Based Position Sizing

In [ ]:
# === 8. Enhancement: Volatility-Based Position Sizing ===
def compute_volatility_size(df, window=30, cap=1.0):
    # Use log returns of aligned SPY price
    price = spy_prices.reindex(df.index)
    log_ret = np.log(price / price.shift(1)).fillna(0)
    rolling_vol = log_ret.rolling(window=window).std()
    avg_vol = rolling_vol.mean()
    size = avg_vol / rolling_vol
    return size.clip(upper=cap)

vol_portfolios = {}

for name, df in model_dfs.items():
    signal = generate_signals(df)
    size = compute_volatility_size(df)
    aligned_spy = spy_prices.reindex(df.index)
    pf = run_vectorbt_strategy(df, signal, aligned_spy, size=size)
    vol_portfolios[name] = pf

# Add SPY Buy & Hold (no sizing applied)
aligned_spy = spy_prices.reindex(df_lstm.index)
spy_pf = vbt.Portfolio.from_signals(
    close=aligned_spy,
    entries=pd.Series(True, index=aligned_spy.index),
    exits=pd.Series(False, index=aligned_spy.index),
    freq='1D',
    init_cash=100,
    fees=0.0
)
vol_portfolios["SPY (Benchmark)"] = spy_pf
    

## Plot Volatility-Based Position Sizing Strategy Performance

In [ ]:
# === Plot Portfolio Values with Volatility-Based Sizing ===
plt.figure(figsize=(12, 6))

for name, pf in vol_portfolios.items():
    model_key = name.replace(" + VolSizing", "")
    color = model_colors.get(model_key, "#333333")  # fallback to dark gray
    label = name if name == "SPY (Benchmark)" else f"{model_key} + VolSizing"
    plt.plot(pf.value(), label=label, linewidth=2.5, alpha=0.9, color=color)

plt.title("Model Strategies with Volatility-Based Sizing", fontsize=16, fontweight='bold')
plt.xlabel("Date", fontsize=14, fontweight='bold')
plt.ylabel("Portfolio Value (Base 100)", fontsize=14, fontweight='bold')

# Bold tick labels
plt.xticks(fontsize=11, fontweight='bold')
plt.yticks(fontsize=11, fontweight='bold')

# Legend styling
legend = plt.legend(fontsize=12, loc='best', frameon=True)
for text in legend.get_texts():
    text.set_fontweight('bold')

plt.grid(True, linestyle='--', alpha=0.7)
plt.tight_layout()
plt.savefig("2_volatility_sizing_comparison.png", dpi=150, bbox_inches='tight')
plt.show()


### Volatility-Based Position Sizing Summary Table

In [ ]:
# === Summary Table for Volatility-Sized Strategies ===
summary = pd.DataFrame()

for name, pf in vol_portfolios.items():
    stats = {
        'Final Value': pf.value().iloc[-1],
        'Total Return [%]': pf.total_return() * 100,
        'Sharpe Ratio': pf.sharpe_ratio(),
        'Max Drawdown [%]': pf.max_drawdown() * 100,
        'Trades': pf.stats()['Total Trades']
    }
    summary[name] = pd.Series(stats)

summary = summary.T.round(2)
display(summary.sort_values('Sharpe Ratio', ascending=False))


## 9: Rolling Stop-Loss / Drawdown-Based Exit

### 9.1 Define Stop-Loss Logic

In [ ]:
def rolling_drawdown_exit(price_series, window=30, threshold=0.05):
    rolling_max = price_series.rolling(window).max()
    drawdown = (rolling_max - price_series) / rolling_max
    exit_signal = drawdown > threshold
    return exit_signal.astype(int)


### 9.2 Run Stop-Loss Strategy per Model

In [ ]:
stoploss_portfolios = {}

for name, df in model_dfs.items():
    signal = generate_signals(df)
    aligned_spy = spy_prices.reindex(df.index)
    exit_signal = rolling_drawdown_exit(aligned_spy)
    
    pf = vbt.Portfolio.from_signals(
        close=aligned_spy,
        entries=signal == 1,
        exits=exit_signal == 1,
        size=1.0,
        freq='1D',
        init_cash=100,
        fees=0.001
    )
    stoploss_portfolios[name] = pf


# === Add SPY Buy & Hold to StopLoss Portfolio Dict for Comparison ===
aligned_spy = spy_prices.reindex(df_lstm.index)

spy_pf = vbt.Portfolio.from_signals(
    close=aligned_spy,
    entries=pd.Series(True, index=aligned_spy.index),
    exits=pd.Series(False, index=aligned_spy.index),
    freq='1D',
    init_cash=100,
    fees=0.0
)

stoploss_portfolios["SPY (Benchmark)"] = spy_pf

### 9.3 Plot Stop-Loss Strategy Performance

In [ ]:
plt.figure(figsize=(12, 6))

# Plot all vectorbt portfolios consistently (including SPY)
for name, pf in stoploss_portfolios.items():
    model_key = name.replace(" + StopLoss", "")
    color = model_colors.get(model_key, "#333333")  # fallback to dark gray
    label = name if name == "SPY (Benchmark)" else f"{model_key} + StopLoss"
    plt.plot(pf.value(), label=label, linewidth=2.5, alpha=0.9, color=color)

plt.title("Model Strategies with Rolling Drawdown Stop-Loss", fontsize=16, fontweight='bold')
plt.xlabel("Date", fontsize=14, fontweight='bold')
plt.ylabel("Portfolio Value (Base 100)", fontsize=14, fontweight='bold')

# Bold axis ticks
plt.xticks(fontsize=11, fontweight='bold')
plt.yticks(fontsize=11, fontweight='bold')

# Styled legend
legend = plt.legend(fontsize=12, loc='best', frameon=True)
for text in legend.get_texts():
    text.set_fontweight('bold')

plt.grid(True, linestyle='--', alpha=0.7)
plt.tight_layout()
plt.savefig("3_stoploss_comparison.png", dpi=150)
plt.show()


### 9.4 Stop-Loss Summary Table

In [ ]:
summary = pd.DataFrame()

for name, pf in stoploss_portfolios.items():
    stats = {
        'Final Value': pf.value().iloc[-1],
        'Total Return [%]': pf.total_return() * 100,
        'Sharpe Ratio': pf.sharpe_ratio(),
        'Max Drawdown [%]': pf.max_drawdown() * 100,
        'Trades': pf.stats()['Total Trades']
    }
    summary[name] = pd.Series(stats)

summary = summary.T.round(2)
display(summary.sort_values('Sharpe Ratio', ascending=False))


## Section 10: Hybrid Strategy

We’ll combine:

- Min-Hold Constraint

- Volatility-Based Position Sizing

- Rolling Drawdown-Based Stop-Loss

### 10.1 Generate Signal + Apply Holding Rule

In [ ]:
# === 10.1 Generate Signal + Apply Holding Rule ===
hybrid_signals = {}

for name, df in model_dfs.items():
    raw_signal = generate_signals(df)
    held_signal = apply_min_holding(raw_signal, min_hold=3)
    hybrid_signals[name] = held_signal


### 10.2 Compute Volatility-Based Position Sizing

In [ ]:
# === 10.2 Compute Volatility-Based Position Sizing ===
hybrid_sizes = {}

for name, df in model_dfs.items():
    size = compute_volatility_size(df)
    hybrid_sizes[name] = size


### 10.3 Compute Rolling Drawdown Stop-Loss Exit

In [ ]:
# === 10.3 Compute Rolling Drawdown Stop-Loss Exit ===
hybrid_exits = {}

for name, df in model_dfs.items():
    aligned_spy = spy_prices.reindex(df.index)
    exit_signal = rolling_drawdown_exit(aligned_spy)
    hybrid_exits[name] = exit_signal


### 10.4 Run Vectorbt Hybrid Backtest

In [ ]:
# === 10.4 Run Vectorbt Hybrid Backtests ===
hybrid_portfolios = {}

for name, df in model_dfs.items():
    aligned_spy = spy_prices.reindex(df.index)
    pf = vbt.Portfolio.from_signals(
        close=aligned_spy,
        entries=hybrid_signals[name] == 1,
        exits=hybrid_exits[name] == 1,
        size=hybrid_sizes[name],
        freq='1D',
        init_cash=100,
        fees=0.001
    )
    hybrid_portfolios[name] = pf


### 10.5 Add SPY Benchmark Portfolio (No Enhancements)

In [ ]:
# === 10.5 Add SPY Benchmark Portfolio (No Enhancements) ===
aligned_spy = spy_prices.reindex(df_lstm.index)

spy_pf = vbt.Portfolio.from_signals(
    close=aligned_spy,
    entries=pd.Series(True, index=aligned_spy.index),
    exits=pd.Series(False, index=aligned_spy.index),
    freq='1D',
    init_cash=100,
    fees=0.0
)

hybrid_portfolios["SPY (Benchmark)"] = spy_pf


### 10.6 Plot Hybrid Strategy Performance

In [ ]:
# === 10.6 Plot Hybrid Strategy Performance ===
plt.figure(figsize=(12, 6))

for name, pf in hybrid_portfolios.items():
    model_key = name.replace(" + Hybrid", "")
    color = model_colors.get(model_key, "#333333")
    label = name if name == "SPY (Benchmark)" else f"{model_key} + Hybrid"
    plt.plot(pf.value(), label=label, linewidth=2.5, alpha=0.9, color=color)

plt.title("Model Strategies with Hybrid Enhancements (MinHold + Vol Sizing + StopLoss)", fontsize=16, fontweight='bold')
plt.xlabel("Date", fontsize=14, fontweight='bold')
plt.ylabel("Portfolio Value (Base 100)", fontsize=14, fontweight='bold')

# Bold tick labels
plt.xticks(fontsize=11, fontweight='bold')
plt.yticks(fontsize=11, fontweight='bold')

# Bold and clean legend
legend = plt.legend(fontsize=12, loc='best', frameon=True)
for text in legend.get_texts():
    text.set_fontweight('bold')

plt.grid(True, linestyle='--', alpha=0.7)
plt.tight_layout()
plt.savefig("4_hybrid_strategy_comparison.png", dpi=150)
plt.show()


### 10.7 Summary Table: Hybrid Performance Metrics

In [ ]:
# === 10.7 Summary Table: Hybrid Performance Metrics ===
summary = pd.DataFrame()

for name, pf in hybrid_portfolios.items():
    stats = {
        'Final Value': pf.value().iloc[-1],
        'Total Return [%]': pf.total_return() * 100,
        'Sharpe Ratio': pf.sharpe_ratio(),
        'Max Drawdown [%]': pf.max_drawdown() * 100,
        'Trades': pf.stats()['Total Trades']
    }
    summary[name] = pd.Series(stats)

summary = summary.T.round(2)
display(summary.sort_values('Sharpe Ratio', ascending=False))


## Section 11: Final Diagnostics Overview

Including:

- Transaction Cost Sensitivity Re-Comparison

- Turnover & Signal Activity

- Strategy Correlation & Diversification

- Visual diagnostic summaries

### 11.1 Transaction Cost Sensitivity Re-Comparison
In this section, we evaluate how the performance of each hybrid strategy responds to varying transaction costs. This analysis helps assess the robustness of the strategy to realistic trading frictions like slippage, spreads, or commission fees.

We will test the following fee levels:

- 0.00% (idealized)
- 0.05%
- 0.10%
- 0.20%
- 0.50%

This analysis is run on the full hybrid strategy (MinHold + Volatility Sizing + StopLoss).

In [ ]:
fee_levels = [0.0, 0.0005, 0.001, 0.002, 0.005]
sensitivity_results = {}

for fee in fee_levels:
    portfolios = {}
    for name, df in model_dfs.items():
        signal = apply_min_holding(generate_signals(df), min_hold=3)
        size = compute_volatility_size(df)
        aligned_spy = spy_prices.reindex(df.index)
        exit_signal = rolling_drawdown_exit(aligned_spy)

        pf = vbt.Portfolio.from_signals(
            close=aligned_spy,
            entries=signal == 1,
            exits=exit_signal == 1,
            size=size,
            freq='1D',
            init_cash=100,
            fees=fee
        )
        portfolios[name] = pf
    sensitivity_results[fee] = portfolios


### 11.2 Performance Sensitivity to Transaction Costs (Sharpe Ratio)

We now visualize how Sharpe Ratios vary for each hybrid strategy across different transaction cost assumptions. This helps us identify which models degrade gracefully under trading frictions — a key requirement for live deployment.


In [ ]:
# === 11.2 Plot Sharpe Ratio Sensitivity Across Fee Levels ===
import matplotlib.pyplot as plt

# Build a DataFrame: rows = models, cols = fees
sharpe_matrix = pd.DataFrame(index=model_dfs.keys(), columns=fee_levels)

for fee in fee_levels:
    for model in model_dfs.keys():
        pf = sensitivity_results[fee][model]
        sharpe_matrix.loc[model, fee] = pf.sharpe_ratio()

sharpe_matrix = sharpe_matrix.astype(float)

# Plot
plt.figure(figsize=(12, 6))
for model in sharpe_matrix.index:
    plt.plot(sharpe_matrix.columns, sharpe_matrix.loc[model], 
             label=model, marker='o', linewidth=2.5, 
             color=model_colors.get(model, "#333333"))

plt.title("Sharpe Ratio Sensitivity Across Transaction Costs", fontsize=16, fontweight='bold')
plt.xlabel("Transaction Cost (per trade)", fontsize=14, fontweight='bold')
plt.ylabel("Sharpe Ratio", fontsize=14, fontweight='bold')
plt.xticks(fontsize=11, fontweight='bold')
plt.yticks(fontsize=11, fontweight='bold')

legend = plt.legend(title="Model", fontsize=11, frameon=True)
for text in legend.get_texts():
    text.set_fontweight('bold')

plt.grid(True, linestyle='--', alpha=0.6)
plt.tight_layout()
plt.savefig("5_sharpe_vs_fee_sensitivity.png", dpi=150)
plt.show()


### 11.2.1 Heatmap of Sharpe Ratios (Model × Transaction Cost)

This heatmap summarizes the impact of transaction costs on strategy robustness. Darker colors indicate lower Sharpe Ratios. More resilient strategies will maintain brighter regions even as costs rise.


In [ ]:
import seaborn as sns

# Reuse the same `sharpe_matrix` DataFrame (models × fees)
plt.figure(figsize=(10, 6))
sns.heatmap(
    sharpe_matrix,
    annot=True, fmt=".2f", cmap="RdYlGn", linewidths=0.5,
    cbar_kws={'label': 'Sharpe Ratio'}
)

plt.title("Sharpe Ratio Heatmap Across Transaction Costs", fontsize=16, fontweight='bold')
plt.xlabel("Transaction Cost", fontsize=14, fontweight='bold')
plt.ylabel("Model", fontsize=14, fontweight='bold')
plt.xticks(fontsize=11, fontweight='bold')
plt.yticks(fontsize=11, fontweight='bold', rotation=0)
plt.tight_layout()
plt.savefig("6_sharpe_heatmap_fee_sensitivity.png", dpi=150)
plt.show()


### 11.3 Turnover & Signal Activity

We analyze how frequently each strategy trades. High turnover can lead to higher slippage, frictional costs, and tax inefficiency. This section computes total trades and visualizes signal activity for hybrid strategies at a baseline fee level (0.001).


In [ ]:
# === 11.3 Total Trades at Base Fee (0.001) ===
base_fee = 0.001
turnover_stats = {}

for name, pf in sensitivity_results[base_fee].items():
    stats = pf.stats()

    turnover_stats[name] = {
        'Total Trades': stats.get('Total Trades', np.nan),
        'Avg Holding Period': stats.get('Avg Holding Period', np.nan),
        'Exposure Time [%]': stats.get('Exposure Time [%]', np.nan)
    }

turnover_df = pd.DataFrame(turnover_stats).T.round(2)
display(turnover_df.sort_values('Total Trades', ascending=False))



#### Signal Switching Frequency (MinHold-Adjusted)

Since most hybrid strategies exhibit very few actual trades due to conservative logic (e.g., holding periods, stop-loss exit filters), we use **signal transition frequency** as a proxy for activity.

This reflects how often each model flips its regime signal from 0 → 1 or 1 → 0 (after minimum holding enforcement). It highlights which models are more reactive to short-term changes.

CNN-LSTM, for example, shows very high signal activity compared to ATT-LSTM, indicating potentially higher turnover if not constrained by post-signal filters.


In [ ]:
# === Signal Activity Diagnostic: Count Switches ===
signal_flips = {}

for name, df in model_dfs.items():
    signal = apply_min_holding(generate_signals(df), min_hold=3)
    flips = (signal != signal.shift(1)).sum()
    signal_flips[name] = flips

signal_flips_series = pd.Series(signal_flips).sort_values(ascending=False)
display(signal_flips_series)


### Plot: Signal Switching Frequency

In [ ]:
plt.figure(figsize=(10, 5))
signal_flips_series.plot(kind='bar', color=[model_colors.get(m, "#888888") for m in signal_flips_series.index])

plt.title("Signal Switching Frequency", fontsize=16, fontweight='bold')
plt.ylabel("Number of Signal Transitions", fontsize=14, fontweight='bold')
plt.xticks(rotation=45, fontsize=11, fontweight='bold')
plt.yticks(fontsize=11, fontweight='bold')
plt.grid(axis='y', linestyle='--', alpha=0.6)
plt.tight_layout()
plt.savefig("7_signal_activity_by_model.png", dpi=150)
plt.show()


## 11.4 Strategy Correlation & Diversification

Understanding correlation between model outputs helps determine whether combining strategies offers diversification benefits — or whether they’re redundant.

In this section, we compute:

- **Signal Correlation**: Correlation between post-processed binary signals (after MinHold).
- **Return Correlation**: Correlation between realized daily returns from hybrid strategies.


### 11.4.1 Signal Correlation Matrix (based on 0/1 signals)

In [ ]:
# === 11.4.1 Signal Correlation Matrix (MinHold Adjusted) ===
signal_matrix = pd.DataFrame(index=df_lstm.index)

for name, df in model_dfs.items():
    signal = apply_min_holding(generate_signals(df), min_hold=3)
    signal_matrix[name] = signal

signal_corr = signal_matrix.corr()

# Display correlation matrix
display(signal_corr.round(2))


#### Heatmap: Signal Correlation

In [ ]:
plt.figure(figsize=(8, 6))
sns.heatmap(signal_corr, annot=True, cmap="coolwarm", fmt=".2f", linewidths=0.5,
            cbar_kws={"label": "Correlation"}, square=True)

plt.title("Signal Correlation Matrix", fontsize=16, fontweight='bold')
plt.xticks(fontsize=11, fontweight='bold', rotation=45)
plt.yticks(fontsize=11, fontweight='bold', rotation=0)
plt.tight_layout()
plt.savefig("8_signal_correlation_heatmap.png", dpi=150)
plt.show()


#### Interpretation: Signal Correlation Matrix

| Insight                        | Observation                                                                 |
|-------------------------------|------------------------------------------------------------------------------|
| LSTM–GRU / LSTM–CNN-LSTM    | Moderate overlap (~0.4) — likely due to similar architecture backbone        |
| ATT-LSTM & Transformer      | Show **lowest correlations** with others (~0.17–0.20) — great for diversification |
| Transformer                 | Least correlated model — strongest candidate for **ensemble diversification** |
| CNN-LSTM                    | Moderate overlap with all, but never strongly dominant                        |

**Conclusion**: The signal behaviors across models are distinct enough to justify diversification via signal blending or ensemble voting.


#### 11.4.2 Return Correlation Matrix (Hybrid Strategy Returns)

We compute correlation between daily returns of each hybrid strategy. This is a practical diversification test — if return correlation is low, combining these models can reduce overall portfolio volatility.


#### Return Correlation Matrix

In [ ]:
# === 11.4.2 Return Correlation Matrix from Hybrid Portfolios ===
return_matrix = pd.DataFrame()

for name, pf in hybrid_portfolios.items():
    if name != "SPY (Benchmark)":
        returns = pf.returns().fillna(0)
        return_matrix[name] = returns

return_corr = return_matrix.corr()

# Display the correlation matrix
display(return_corr.round(2))


#### Heatmap: Return Correlation

In [ ]:
plt.figure(figsize=(8, 6))
sns.heatmap(return_corr, annot=True, cmap="coolwarm", fmt=".2f", linewidths=0.5,
            cbar_kws={"label": "Correlation"}, square=True)

plt.title("Return Correlation Matrix (Hybrid Strategies)", fontsize=16, fontweight='bold')
plt.xticks(fontsize=11, fontweight='bold', rotation=45)
plt.yticks(fontsize=11, fontweight='bold', rotation=0)
plt.tight_layout()
plt.savefig("9_return_correlation_heatmap.png", dpi=150)
plt.show()


#### Interpretation: Return Correlation Matrix

| Insight                          | Observation                                                                 |
|----------------------------------|------------------------------------------------------------------------------|
| High Overall Correlation      | All models have return correlations > 0.95 — indicating tight co-movement. |
| ATT-LSTM & Transformer        | Almost perfectly aligned with LSTM/each other (~0.99) despite lower signal correlation. |
| CNN-LSTM Slightly Differentiated | CNN-LSTM shows slightly lower return correlation (~0.95–0.96), but still highly aligned. |
| Signal ≠ Return               | This highlights how different signals can **converge in economic impact**, especially under shared market exposure. |

**Conclusion**: Diversification across these hybrid models may offer more value in **timing and signal mixing** than in reducing overall portfolio volatility.


## 12. Final Conclusions & Strategy Summary

We consolidate the results of our hybrid strategy evaluation under baseline transaction costs (`fee = 0.001`). This summary compares key performance metrics across models.

We also reflect on robustness across enhancements, cost sensitivity, signal efficiency, and diversification potential.


In [ ]:
# === 12. Final Summary Table ===
summary_table = pd.DataFrame()

for name, pf in hybrid_portfolios.items():
    if name != "SPY (Benchmark)":
        stats = pf.stats()
        summary_table[name] = {
            'Final Value': pf.value().iloc[-1],
            'Total Return [%]': pf.total_return() * 100,
            'Sharpe Ratio': pf.sharpe_ratio(),
            'Max Drawdown [%]': pf.max_drawdown() * 100,
            'Total Trades': stats.get('Total Trades', np.nan)
        }

summary_table = summary_table.T.round(2)
summary_table = summary_table.sort_values('Sharpe Ratio', ascending=False)

display(summary_table)


### Key Takeaways

- **Top Sharpe Model**: ATT-LSTM / LSTM show consistently strong risk-adjusted performance.
- **Lowest Signal Overlap**: Transformer, suggesting diversification potential.
- **Most Robust to Costs**: GRU and ATT-LSTM.
- **Most Active**: CNN-LSTM → higher switching, more cost sensitive.

These results can inform model ensembling, regime-based switching, or even adaptive weighting schemes in portfolio applications.


## Strategy Comparison Radar Chart: Normalized Metrics

This radar chart compares all hybrid strategies across five key metrics, each normalized on a 0–1 scale:

- **Final Value**
- **Total Return [%]**
- **Sharpe Ratio**
- **Max Drawdown [%]** (inverted so higher = better)
- **Total Trades**

Each model is plotted as a closed polygon. The closer a model reaches the outer edge on all dimensions, the more balanced and dominant its performance.

In [ ]:
from math import pi

# Normalize metrics (0–1 scale) to fairly compare across different units
normalized = summary_table.copy()

# Invert Max Drawdown so higher = better
normalized['Max Drawdown [%]'] = -normalized['Max Drawdown [%]']

# Normalize all columns 0–1
normalized = (normalized - normalized.min()) / (normalized.max() - normalized.min())

# Radar chart requires closing the polygon (repeat first row)
metrics = normalized.columns.tolist()
strategies = normalized.index.tolist()

# Setup plot
plt.figure(figsize=(8, 8))
angles = [n / float(len(metrics)) * 2 * pi for n in range(len(metrics))]
angles += angles[:1]  # close the circle

for strategy in strategies:
    values = normalized.loc[strategy].tolist()
    values += values[:1]  # close the circle
    plt.polar(angles, values, label=strategy, linewidth=2, alpha=0.7)

# Add labels
plt.xticks(angles[:-1], metrics, fontsize=12, fontweight='bold')
plt.title("Strategy Comparison Radar Chart (Normalized Metrics)", fontsize=16, fontweight='bold')
plt.yticks([0.25, 0.5, 0.75, 1.0], ["25%", "50%", "75%", "100%"], fontsize=10)

# Legend fix
legend = plt.legend(loc='upper right', bbox_to_anchor=(1.2, 0.99), frameon=True, fontsize=11)
for text in legend.get_texts():
    text.set_fontweight('bold')

plt.tight_layout()
plt.savefig("10_strategy_radar_chart.png", dpi=150)
plt.show()


#### Key Observations:

| Strategy    | Strengths                                      | Weaknesses               |
|-------------|------------------------------------------------|---------------------------|
| **LSTM**    | Top in Total Return, Final Value, Sharpe       | Slightly higher Drawdown |
| **ATT-LSTM**| Very strong Sharpe + return profile            | Fewer trades              |
| **GRU**     | Balanced performer, most robust to transaction costs | Mid-tier return          |
| **Transformer** | Strong diversification potential, good Sharpe | Less aggressive returns  |
| **CNN-LSTM**| Highest signal activity, moderate performance  | Weak on Sharpe and Drawdown |

 
 **Conclusion**: LSTM and ATT-LSTM dominate most dimensions, while GRU and Transformer offer stability and diversification. CNN-LSTM may be best used as a tactical overlay or signal enhancer.